In [7]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD

In [8]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json',encoding='utf-8').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?', '!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hi'], 'introduction'), (['Hello'], 'introduction'), (['Aap', 'kya', 'kar', 'sakte', 'ho', '?'], 'introduction'), (['Namaste'], 'introduction'), (['Kya', 'aap', 'meri', 'help', 'kar', 'sakte', 'ho', '?'], 'introduction'), (['Main', 'nursing', 'college', 'mein', 'admission', 'lena', 'chahta', 'hoon'], 'admission_interest'), (['Mujhe', 'nursing', 'program', 'ke', 'liye', 'apply', 'karna', 'hai'], 'admission_interest'), (['Admission', 'ke', 'liye', 'kya', 'karna', 'hoga'], 'admission_interest'), (['B.Sc', 'Nursing', 'ka', 'admission', 'chahiye'], 'admission_interest'), (['How', 'can', 'I', 'get', 'admission', 'in', 'nursing', 'college'], 'admission_interest'), (['Haan', 'maine', '12th', 'mein', 'biology', 'padha', 'hai'], 'eligibility_check'), (['Yes', 'I', 'had', 'biology', 'in', 'class', '12'], 'eligibility_check'), (['Mera', '12th', 'mein', 'subject', 'biology', 'tha'], 'eligibility_check'), (['Biology', 'tha', 'mere', 'paas', '12th', 'mein'], 'eligibility_check'), (['Nahi', 'maine'

In [9]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

['12', '12th', 'Aap', 'Accommodation', 'Admission', 'B.Sc', 'Biology', 'Clinical', 'College', 'Course', 'Delhi', 'Eligibility', 'Fee', 'Financial', 'Haan', 'Hello', 'Hi', 'Hospital', 'Hostel', 'How', 'I', 'INC', 'Is', 'Kitna', 'Kya', 'Location', 'Main', 'Maine', 'Mera', 'Mujhe', 'Nahi', 'Namaste', 'No', 'Nursing', 'Recognition', 'Scholarship', 'Seats', 'Tell', 'Total', 'Training', 'What', 'Where', 'Who', 'Yes', 'aap', 'aayega', 'about', 'admission', 'aid', 'apply', 'approval', 'approved', 'aur', 'availability', 'available', 'baare', 'batao', 'batayein', 'biology', 'can', 'center', 'chahiye', 'chahta', 'class', 'college', 'commerce', 'concession', 'condition', 'council', 'course', 'criterion', 'detail', 'did', 'dijiye', 'eligibility', 'facility', 'fee', 'for', 'get', 'had', 'hai', 'hain', 'help', 'ho', 'hoga', 'hoon', 'hospital', 'hote', 'hoti', 'in', 'is', 'ka', 'kahan', 'kar', 'karna', 'kaun', 'ke', 'kharcha', 'ki', 'kitne', 'kitni', 'koi', 'kya', 'lena', 'list', 'liya', 'liye', 'main

In [10]:
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [11]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag=[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row= list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size =5, verbose=1)

model.save('chatbot_model.h5', hist)
print("Done")

Epoch 1/200
11/11 [==============================] - 1s 5ms/step - loss: 2.6004 - accuracy: 0.1296
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 2.5028 - accuracy: 0.1296
Epoch 3/200
11/11 [==============================] - 0s 3ms/step - loss: 2.4753 - accuracy: 0.1296
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 2.4136 - accuracy: 0.2963
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 2.3572 - accuracy: 0.2963
Epoch 6/200
11/11 [==============================] - 0s 4ms/step - loss: 2.3191 - accuracy: 0.2407
Epoch 7/200
11/11 [==============================] - 0s 5ms/step - loss: 2.1792 - accuracy: 0.2593
Epoch 8/200
11/11 [==============================] - 0s 5ms/step - loss: 2.0770 - accuracy: 0.3519
Epoch 9/200
11/11 [==============================] - 0s 4ms/step - loss: 2.0538 - accuracy: 0.4074
Epoch 10/200
11/11 [==============================] - 0s 3ms/step - loss: 1.8431 - accuracy: 0.4815
Epoch 11/